In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [9]:
def tokenizer(text):
  text = text.lower()
  text = text.replace("?",'')
  text = text.replace('"','')
  text = text.replace("'",'')
  return text.split()

In [11]:
tokenizer('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [12]:
vocab = {'<UNK>':0}

In [21]:
def build_vocab(row):
  tokenized_question = tokenizer(row['question'])
  tokenized_answer = tokenizer(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

In [22]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [23]:
len(vocab)

326

In [34]:
def text_to_indices(text, vocab):

  index_text = []

  for token in tokenizer(text):
    if token in vocab:
      index_text.append(vocab[token])
    else:
      index_text.append(vocab['<UNK>'])

  return index_text

In [37]:
text_to_indices("What is campusx", vocab)

[3, 4, 0]

In [38]:
import torch
from torch.utils.data import Dataset, DataLoader

In [39]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [40]:
dataset = QADataset(df, vocab)

In [41]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [44]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[  3,   4,   5,  94, 139,  21,   5,  47]]) tensor([187])
tensor([[  3,   4,   5,   6,   7, 208]]) tensor([209])
tensor([[ 3,  4,  5, 26, 27,  7, 28, 21, 29]]) tensor([30])
tensor([[ 3,  4,  5, 94, 95, 96]]) tensor([97])
tensor([[  3,   4,   5, 105,   7, 106,  21, 107]]) tensor([108])
tensor([[12, 98,  5, 99]]) tensor([100])
tensor([[ 12,   4,  64,  65,   5, 285,   7, 286]]) tensor([287])
tensor([[ 12,  77, 113]]) tensor([114])
tensor([[ 80,  81, 263, 153,  16, 264, 155]]) tensor([38])
tensor([[  3,   4,   5,  35,  36,   7, 247]]) tensor([248])
tensor([[ 3,  4,  5,  6,  7, 55]]) tensor([56])
tensor([[  3,   4,   5, 236,   7, 237]]) tensor([133])
tensor([[  3,   4,   5, 182, 183, 184, 185]]) tensor([186])
tensor([[ 12,  13, 191, 160, 192]]) tensor([193])
tensor([[ 80,  81, 197,  83,  21,   5, 198, 199, 200]]) tensor([201])
tensor([[12, 13, 14, 15, 16, 17]]) tensor([18])
tensor([[ 12,  77, 210]]) tensor([211])
tensor([[ 80,  81, 131,  83,  21,   5,  23,  24]]) tensor([38])
tensor(

In [45]:
import torch.nn as nn

In [51]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [52]:
learning_rate = 0.001
epochs = 20

model = SimpleRNN(len(vocab))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [53]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 527.053850
Epoch: 2, Loss: 453.849874
Epoch: 3, Loss: 373.945892
Epoch: 4, Loss: 315.174096
Epoch: 5, Loss: 264.251416
Epoch: 6, Loss: 214.712472
Epoch: 7, Loss: 171.086889
Epoch: 8, Loss: 133.552125
Epoch: 9, Loss: 102.143108
Epoch: 10, Loss: 78.896223
Epoch: 11, Loss: 61.202485
Epoch: 12, Loss: 48.418018
Epoch: 13, Loss: 39.113270
Epoch: 14, Loss: 31.619970
Epoch: 15, Loss: 25.974737
Epoch: 16, Loss: 21.986091
Epoch: 17, Loss: 18.712555
Epoch: 18, Loss: 15.923685
Epoch: 19, Loss: 13.714796
Epoch: 20, Loss: 11.778522


In [54]:
def predict(model, question, threshold=0.5):

  numerical_question = text_to_indices(question, vocab)

  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  output = model(question_tensor)

  probs = torch.nn.functional.softmax(output, dim=1)

  value, index = torch.max(probs, dim=1)

  if value<threshold:
    print('I dont know')

  print(list(vocab.keys())[index])

In [55]:
predict(model, "What is the largest planet in our solar system?")

jupiter
